# TAREA 2
## Modelos de lenguage: N-gramas
### Parte 1

A continuación se realizan las importaciones necesarias para importar bibliotecas para el procesamiento de lenguaje natural con NLTK, manejo de archivos comprimidos, web scraping, manipulación de datos JSON y combinaciones, además de optimización de cálculos usando GPU con cupy. Facilita tareas como tokenización, creación de n-gramas y análisis eficiente de datos textuales.

In [6]:
import os
import re
from collections import Counter
import nltk
from nltk import (
sent_tokenize,
word_tokenize,
ngrams,
bigrams,
trigrams
)
import tarfile
import lxml
from bs4 import BeautifulSoup
import requests
import random
import zipfile
import json
from itertools import combinations_with_replacement
import cupy as cp

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


A continuación, se descargan los recursos de NLTK para permitir la tokenización de texto:

*nltk.download('punkt'):* Descarga el modelo necesario para dividir texto en oraciones y palabras.    
*nltk.download('punkt_tab'):* Mejora la precisión de la tokenización.

Ambas son clave para procesar los textos y construir los modelos N-Gram en el ejercicio.

In [7]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

A continuación, se establece una variable llamada DATASETS_DIR que almacena la ruta del directorio donde se encuentran los conjuntos de datos (datasets) que se utilizarán en el ejercicio, en este caso, el directorio se llama DATASETS/.

In [12]:
# Esta es la ruta definida para trabajar con los datasets
DATSETS_DIR = "C:/Users/LENOVO/Downloads/Datasets"

Estas funciones extraen archivos comprimidos, esenciales para descomprimir los datasets (20N y BAC) en el ejercicio, permiten acceder a los datos necesarios para procesar y construir los modelos de N-Gram.

In [13]:
def extract_tarfile(file_path, extract_to):
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=extract_to)

def extract_zipfile(file_path, extract_to):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

La siguiente función extrae archivos zip dentro de otros zip de manera recursiva, asegurando que todos los datasets (20N y BAC) estén completamente descomprimidos, incluso si están anidados, para poder procesarlos y construir los modelos N-Gram.

1. Recorre un directorio buscando archivos zip.
2. Los extrae y elimina el archivo zip original.
3. Si hay más archivos zip dentro, repite el proceso.


In [14]:
import time
def extract_nested_zips(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.zip'):
              nested_zip_path = os.path.join(root, filename)
              print(f"Extracting nested zip: {nested_zip_path}")
              with zipfile.ZipFile(nested_zip_path, 'r') as nested_zip_ref:
                  nested_zip_ref.extractall(root)
              if os.path.exists(nested_zip_path):
                  time.sleep(5)
                  os.remove(nested_zip_path)
              extract_nested_zips(root)

### 1. (5p) Read the files and build two large consolidate files that are the union of all the documents in 20N and BAC.

A continuación, se leen y combinan todos los archivos del dataset 20Newsgroups en un solo archivo de texto, aquí se prepara el dataset consolidando todos los textos, lo que es esencial para la creación de los modelos N-Gram.

1. Se extrae el archivo comprimido del dataset.
2. Se leen todos los archivos en el directorio y los une en una sola cadena de texto.
3. Se guarda ese texto consolidado en un archivo de salida.


In [15]:
def read_20newsgroups_files(directory, output_file):
    extract_tarfile(os.path.join(DATSETS_DIR, '20news-18828.tar.gz'), '20Newsgroups/')
    consolidated_text = ""
    for root, dirs, files in os.walk(directory):
        for filename in files:
            with open(os.path.join(root, filename), 'r', encoding='latin-1') as file:
                consolidated_text += file.read() + " "
    with open(output_file, 'w', encoding='utf-8') as out_file:
        out_file.write(consolidated_text)


    return consolidated_text

newsgroups_text = read_20newsgroups_files('20Newsgroups/','20NewsGroupConsolidated.txt' )

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6180\2326178727.py:3: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_to)


A continuación, se procesan los archivos del dataset Blog Authorship Corpus (BAC), aquí se prepara el dataset BAC consolidando el texto necesario para crear los modelos N-Gram. Esta implementación extrae archivos comprimidos de BAC si es necesario, procesa los archivos .xml para extraer el contenido de las etiquetas <post>, y consolida todo el texto en un archivo llamado BCAConsolidated.txt. También elimina duplicados si los encuentra.

In [21]:
import os

def extract_tarfile_if_needed(tar_path, extract_to):
    """ Extrae el archivo TAR si aún no ha sido extraído. """
    if not os.path.exists(extract_to):
        print(f"Extracting {tar_path} to {extract_to}")
        extract_tarfile(tar_path, extract_to)
    else:
        print(f"{extract_to} already exists, skipping extraction.")

def extract_zipfile_if_needed(zip_path, extract_to):
    """ Extrae el archivo ZIP si aún no ha sido extraído. """
    if not os.path.exists(extract_to):
        print(f"Extracting {zip_path} to {extract_to}")
        extract_zipfile(zip_path, extract_to)
    else:
        print(f"{extract_to} already exists, skipping extraction.")

def read_BCA_files(directory, output_file):
    bac_zip_path = os.path.join(DATSETS_DIR, 'BAC.zip')
    bac_tar_path = os.path.join(DATSETS_DIR, 'BAC.tar.gz')

    if os.path.exists(bac_zip_path):
        extract_zipfile_if_needed(bac_zip_path, directory)
    elif os.path.exists(bac_tar_path):
        extract_tarfile_if_needed(bac_tar_path, directory)
    
    blogs_zip_path = os.path.join(directory, 'blogs.zip')
    blogs_extract_path = os.path.join(directory, 'blogs')
    
    extract_zipfile_if_needed(blogs_zip_path, blogs_extract_path)
    
    consolidated_text = ""
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.xml'):
                if "blogs" not in f"{root}/{filename}":
                    print(f"{directory}/blogs")
                    print(os.listdir(f"{directory}/blogs"))
                    print(filename in os.listdir(f"{directory}/blogs"))
                    if filename in os.listdir(f"{directory}/blogs"):
                        os.remove(f"{root}/{filename}")
                        continue
                
                xml_file_path = os.path.join(root, filename)
                print(f"Reading XML file: {xml_file_path}")
                try:
                    with open(xml_file_path, 'r', encoding='ISO-8859-1') as xml_file:
                        content = xml_file.read()
                    soup = BeautifulSoup(content, 'lxml')
                    post_element = soup.find('post')
                    if post_element:
                        consolidated_text += post_element.get_text(strip=True) + "\n"
                except Exception as e:
                    print(f"Error processing file {xml_file_path}: {e}")
    
    with open(output_file, 'w', encoding='utf-8') as out_file:
        out_file.write(consolidated_text)
    return consolidated_text

blogs_text = read_BCA_files(os.path.join(DATSETS_DIR, 'BAC'), 'BCAConsolidated.txt')

Extracting C:/Users/LENOVO/Downloads/Datasets\BAC\blogs.zip to C:/Users/LENOVO/Downloads/Datasets\BAC\blogs
C:/Users/LENOVO/Downloads/Datasets\BAC/blogs
['blogs']
False
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\5114.male.25.indUnk.Scorpio.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\1000331.female.37.indUnk.Leo.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\1000866.female.17.Student.Libra.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\1004904.male.23.Arts.Capricorn.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\1005076.female.25.Arts.Cancer.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\1005545.male.25.Engineering.Sagittarius.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\1007188.male.48.Religion.Libra.xml
Reading XML file: C:/Users/LENOVO/Downloads/Datasets\BAC\blogs\blogs\100812.female.26.Architecture.Aries.xml
Re

### 2. (5p) Tokenize by sentence: 

__a. Normalize, but DO NOT eliminate stop words.__

__b. Replace numbers with a token named NUM.__

__c. Add sentence start and end tags <s></s>.__

__d. Tokens with unit frequency should be modeled as <UNK>.__

A continuación, se preprocesa el texto de los datasets 20Newsgroups y BAC, donde se normaliza el texto (minúsculas), se eliminan correos, se reemplaza números con NUM, y se limpian caracteres especiales. Luego, se Tokeniza el texto en oraciones y palabras, añadiendo etiquetas <s> y </s>. Así, estos pasos preparan el corpus para construir los modelos N-Gram necesarios en el ejercicio.

In [22]:
def preprocess_text(text):
    text = text.lower()
    ### Remover correos electrónicos
    text = re.sub(r'[\w\d]+@[\w\d]+\.[\w\d]+\.?[\w\d]*\.?[\w\d]*\.?[\w\d]*\.?[\w\d]*',' ',text)
    #### Reemplazar números con la etiqueta NUM
    text = re.sub(r'\d+', 'NUM', text)
    # Remover con un expresión regular carateres especiales (no palabras) excepto signos de puntuación.
    text = re.sub(r'[^\w\s\.,:;\'\?]', ' ', str(text))

    sentences = sent_tokenize(text)

    tokenized_sentences = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        tokenized_sentences.append(['<s>'] + tokens + ['</s>'])

    return tokenized_sentences

with open('./20NewsGroupConsolidated.txt','r',encoding='utf-8') as file:
    newsgroups_text = file.read()
newsgroups_preprocessed = preprocess_text(newsgroups_text)  ## tokeniza oraciones
with open('./BCAConsolidated.txt','r',encoding='utf-8') as file:
    blogs_text = file.read()

blogs_preprocessed = preprocess_text(blogs_text)        ## tokeniza oraciones

Esta función, reemplaza las palabras raras (frecuencia 1) por <UNK> en los datasets, mejorando la generalización de los modelos N-Gram al manejar palabras raras de forma uniforme. Para ello, cuenta las apariciones de cada token, luego reemplaza los tokens con frecuencia 1 por <UNK>. Posterioremente, aplica esto a las oraciones tokenizadas de 20Newsgroups y BAC.

In [23]:
def handle_unk(tokens_list):
    all_tokens = [token for sentence in tokens_list for token in sentence]
    token_counts = Counter(all_tokens)

    updated_tokens_list = []
    for sentence in tokens_list:
        updated_sentence = [token if token_counts[token] > 1 else '<UNK>' for token in sentence]
        updated_tokens_list.append(updated_sentence)

    return updated_tokens_list

newsgroups_final = handle_unk(newsgroups_preprocessed)
blogs_final = handle_unk(blogs_preprocessed)

### 3. (10p) Select 80% of the resulting sentences -random without replacement-to build the N-gram model and the remaining 20% for evaluation.Create the following files:

**a. 20N_<group_code>_training   (training sentences)**

**b. 20N_<group_code>_testing (testing sentences)**

**c. BAC_<group_code>_training (training sentences)__**

**d. BAC_<group_code>_testing (testing sentences)__**

A continuación, la siguiente implementación divide y guarda los datos preprocesados en conjuntos de entrenamiento y prueba (80%-20%), donde se preparan los datos de 20Newsgroups y BAC para entrenar y evaluar los modelos N-Gram, separando los conjuntos de entrenamiento y prueba. Para ello:

1. La función split_data: Divide las oraciones en 80% para entrenamiento y 20% para prueba.
2. La función save_sentences: Guarda las oraciones divididas en archivos separados para entrenamiento y prueba.


In [35]:
# Crear una carpeta "resultados" en el directorio establecido para el ejercicio
resultados_dir = os.path.join(DATSETS_DIR, 'resultados')
if not os.path.exists(resultados_dir):
    os.makedirs(resultados_dir)

def split_data(sentences, train_ratio=0.8):
    random.shuffle(sentences)
    split_index = int(len(sentences) * train_ratio)
    train_sentences = sentences[:split_index]
    test_sentences = sentences[split_index:]
    return train_sentences, test_sentences

def save_sentences(sentences, ctx):
    train_sentences, test_sentences = split_data(sentences)
    for text, name in zip([train_sentences, test_sentences], ["training", "testing"]):
        # Guardar en la carpeta "resultados" con el formato del nombre del grupo (grupo 4)
        file_path = os.path.join(resultados_dir, f"{ctx}_4_{name}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            for sentence in text:
                f.write(' '.join(sentence) + '\n')

# Guardar los conjuntos de entrenamiento y prueba en "resultados"
save_sentences(newsgroups_final, '20N')
save_sentences(blogs_final, 'BAC')

### 4. (50p) Build the following N-gram models using Laplace smoothing and generate an output file for each one (you choose the output structure, but be sure to provide an appropriate pythonreading method/function):

**a. 20N_<group_code>_unigrams**

**b. 20N_<group_code>_bigrams**

**c. 20N_<group_code>_trigrams**

**d. BAC_<group_code>_unigrams**

**e. BAC_<group_code>_bigrams**

**f. BAC_<group_code>_trigram**

Las siguiente rutas, son necesarias para acceder a los datos de entrenamiento que serán utilizados en la construcción de los modelos N-Gram para ambos datasets. Definen dónde están almacenados los archivos con los datos preprocesados y divididos en entrenamiento, facilitando su lectura y uso en los siguientes pasos del ejercicio.

In [40]:
path_training_BAC = './BAC_4_training'
path_training_20N = './20N_4_training'

La función tokenizer_re, tokeniza el texto de entrenamiento usando una expresión regular y devuelve los tokens y el tamaño del vocabulario, prepara los datos para construir los modelos N-Gram, proporcionando los tokens y el tamaño del vocabulario necesarios para calcular probabilidades y aplicar el suavizado Laplace. 

Para ello, lee el archivo de entrenamiento, tokeniza el texto capturando palabras (etiquetas <s>, </s>, y <UNK>), y finalmente, calcula el número de tokens únicos (vocabulario).


In [41]:
def tokenizer_re(path_training, regex_expresion=r'\w+|<s>|</s>|<UNK>'):
    
    text = open(path_training, encoding='utf8').read()
    tokenizer = nltk.RegexpTokenizer(regex_expresion)
    tokens = tokenizer.tokenize(text)
    V = len(set(tokens))  # Tamaño del vocabulario
    return tokens, V

La siguiente implementación, crea un modelo unigrama calculando las probabilidades de cada palabra en el corpus de entrenamiento y guarda los resultados en un archivo JSON, donde se genera el modelo unigrama para los datasets BAC y 20Newsgroups, esencial para construir los modelos N-Gram.

Aqui, se tokeniza el texto y cuenta la frecuencia de las palabras, se calcula la probabilidad de cada palabra dividiendo su frecuencia entre el total de tokens, y finalmente, se guarda el modelo unigrama en un archivo JSON.

In [42]:
def model_unigram(path_training, file_name):
    model_unigram_prob = {}
    tokens, V = tokenizer_re(path_training=path_training)
    frecuencys = Counter(tokens)
    corpus_len = len(tokens)

    for key, value in frecuencys.items():
        model_unigram_prob[key] = (value + 1) / (corpus_len + V)  

    # Guardar en el directorio actual (lógica interna)
    with open(file_name, 'w') as file:
        json.dump(model_unigram_prob, file, indent=4)

    # Guardar una copia en la carpeta "resultados" para lectura posterior
    with open(os.path.join(resultados_dir, file_name), 'w') as file:
        json.dump(model_unigram_prob, file, indent=4)

    return

model_unigram(path_training=path_training_BAC,file_name='BAC_4_unigrams.json')
model_unigram(path_training=path_training_20N,file_name='20N_4_unigrams.json')

A continuación, el código crea un modelo de bigramas con suavizado Laplace y guarda las probabilidades en un archivo JSON, con el fin de generar el modelo bigrama, crucial para los modelos N-Gram, usando suavizado para evitar probabilidades de cero.

Para ello, se tokeniza el texto y cuenta la frecuencia de palabras y bigramas, se calcula la probabilidad de cada bigrama aplicando suavizado Laplace, y finalmente, se guarda el modelo de bigramas en un archivo JSON.

In [43]:
def bigram_model(path, file_name):
    tokens, V = tokenizer_re(path_training=path)
    frecuency_word = Counter(tokens)
    bigrams_prob = {}
    frecuency_bigrams = Counter(list(bigrams(tokens)))

    for bigram, value in frecuency_bigrams.items():
        count_w0 = frecuency_word[bigram[0]] + V  
        frequency_bigram = value + 1  
        bigrams_prob[str(bigram)] = frequency_bigram / count_w0

    # Guardar en el directorio actual (lógica interna)
    with open(file_name, 'w') as file:
        json.dump(bigrams_prob, file, indent=4)

    # Guardar una copia en la carpeta "resultados" para lectura posterior
    with open(os.path.join(resultados_dir, file_name), 'w') as file:
        json.dump(bigrams_prob, file, indent=4)

    return

bigram_model(path=path_training_BAC,file_name='BAC_4_bigrams.json')
bigram_model(path=path_training_20N,file_name='20N_4_bigrams.json')

A continuación, el código crea un modelo de trigramas con suavizado Laplace y guarda las probabilidades en un archivo JSON, donde se genera el modelo trigramas, esencial para los modelos N-Gram, usando suavizado de Laplace para evitar probabilidades cero y mejorar la precisión del modelo.

Para ello, se tokeniza el texto y cuenta las frecuencias de bigramas y trigramas, se calcula la probabilidad de cada trigrama aplicando suavizado Laplace, y finalmente se guarda el modelo de trigramas en un archivo JSON.

In [44]:
def trigram_model(path, file_name):
    tokens, V = tokenizer_re(path_training=path)
    bigrams_frequency = Counter(list(bigrams(tokens)))
    trigrams_frequency = Counter(list(trigrams(tokens)))
    trigram_probs = {}

    for trigram, frequency in trigrams_frequency.items():
        countw0w1w2 = frequency + 1  
        countw0w1 = bigrams_frequency[(trigram[0], trigram[1])] + V  
        trigram_probs[str(trigram)] = countw0w1w2 / countw0w1

    # Guardar en el directorio actual (lógica interna)
    with open(file_name, 'w') as file:
        json.dump(trigram_probs, file, indent=4)

    # Guardar una copia en la carpeta "resultados" para lectura posterior
    with open(os.path.join(resultados_dir, file_name), 'w') as file:
        json.dump(trigram_probs, file, indent=4)

    return

trigram_model(path=path_training_20N,file_name='20N_4_trigrams.json')
trigram_model(path=path_training_BAC,file_name='BAC_4_trigrams.json')